In [1]:
import requests
import pyinputplus as pyip
from pprint import pprint
from requests import Response
from pydantic import BaseModel,RootModel,Field,ValidationError,field_validator
from random import choices

class UB(BaseModel):
    場站中文名稱:str = Field(alias='sna')
    場站區域:str = Field(alias='sarea')
    資料更新時間:str = Field(alias='mday')
    地點:str = Field(alias='ar')
    act:bool
    場站總停車格:int = Field(alias='total')
    場站目前車輛數量:int = Field(alias='available_rent_bikes')
    空位數量:int = Field(alias='available_return_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')

class Root(RootModel):
    root:list[UB]

def main():
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    res:Response = requests.get(youbike_url)
    all_data = res.json()

    try:
        root:Root = Root.model_validate(all_data)
    except ValidationError as error:
            print(error)

    data_list:list[dict] = root.model_dump()
    filter_list:list[dict] = list(filter(lambda item:False if item['act']==0 else True,data_list))
    filter_list_rent_less3:list[dict] = list(filter(lambda item:True if item['場站目前車輛數量']<3 else False,data_list))
    filter_list_return_less3:list[dict] = list(filter(lambda item:True if item['空位數量']<3 else False,data_list))

    print(f"台北市YouBike有{len(data_list)}站點，目前正常運作站點共{len(filter_list)}個")

    num = pyip.inputInt("請輸入隨機顯示的站點數(1~5): ", min=1, max=5)
    print(num)

    print(f"\n可借車輛數為3輛以內的站點有{len(filter_list_rent_less3)}站")
    num_list:list[dict] = choices(filter_list_rent_less3, k=num)
    for sta in num_list:
            seq = filter_list_rent_less3.index(sta)
            print(f"No.{seq}")
            pprint(sta, indent=2, sort_dicts=False)

    print(f"\n可還車輛數為3輛以內的站點有{len(filter_list_return_less3)}站")
    num_list:list[dict] = choices(filter_list_return_less3, k=num)
    for sta in num_list:
            seq = filter_list_return_less3.index(sta)
            print(f"No.{seq}")
            pprint(sta, indent=2, sort_dicts=False)

if __name__ == '__main__':
     main()

    

台北市YouBike有1423站點，目前正常運作站點共1414個
請輸入隨機顯示的站點數(1~5): Number must be at maximum 5.
請輸入隨機顯示的站點數(1~5): 2

可借車輛數為3輛以內的站點有397站
No.215
{ '場站中文名稱': 'YouBike2.0_瑞光公共住宅(瑞光路)',
  '場站區域': '內湖區',
  '資料更新時間': '2024-06-14 18:15:45',
  '地點': '瑞光路160號',
  'act': True,
  '場站總停車格': 18,
  '場站目前車輛數量': 0,
  '空位數量': 18,
  '緯度': 25.07335,
  '經度': 121.57951}
No.52
{ '場站中文名稱': 'YouBike2.0_仁愛復興路口',
  '場站區域': '大安區',
  '資料更新時間': '2024-06-14 18:51:22',
  '地點': '仁愛路四段1號前',
  'act': True,
  '場站總停車格': 25,
  '場站目前車輛數量': 1,
  '空位數量': 24,
  '緯度': 25.03817,
  '經度': 121.544}

可還車輛數為3輛以內的站點有112站
No.62
{ '場站中文名稱': 'YouBike2.0_捷運中山站(3號出口)',
  '場站區域': '中山區',
  '資料更新時間': '2024-06-14 18:50:32',
  '地點': '南京西路1號前',
  'act': True,
  '場站總停車格': 21,
  '場站目前車輛數量': 20,
  '空位數量': 1,
  '緯度': 25.05236,
  '經度': 121.52212}
No.73
{ '場站中文名稱': 'YouBike2.0_堤頂大道新湖一路口',
  '場站區域': '內湖區',
  '資料更新時間': '2024-06-14 18:44:15',
  '地點': '堤頂大道一段215號前',
  'act': True,
  '場站總停車格': 13,
  '場站目前車輛數量': 12,
  '空位數量': 1,
  '緯度': 25.05892,
  '經度': 121.57354}
